In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# pd.set_option('display.max_columns', 30)

### Data

In [2]:
df1 = pd.read_csv('../DATA/clean-HSC-unWISE-W01.csv')
df1 = df1.drop(columns = [f for f in df1.columns if ('isnull' in f)])
df2 = pd.read_csv('../DATA/clean-HSC-unWISE-W02.csv')
df2 = df2.drop(columns = [f for f in df2.columns if ('isnull' in f)])
df = pd.concat([df1,df2], axis = 'rows')
del df1,df2

df['gr'] = df['g_cmodel_mag'] - df['r_cmodel_mag']
df['ri'] = df['r_cmodel_mag'] - df['i_cmodel_mag']
df['iz'] = df['i_cmodel_mag'] - df['z_cmodel_mag']
df['zy'] = df['z_cmodel_mag'] - df['y_cmodel_mag']

with open('../DATA/features1.txt') as file:
    features = file.read().splitlines()
labels = 'member'

n_mem = df[df.member == 1].shape[0]
n_no = df[df.member == 0].shape[0]
n = df.shape[0]
print ('Members: {} ({:.2f}%)'.format(n_mem, n_mem/n*100))
print ('Non members: {} ({:.2f}%)'.format(n_no, n_no/n*100))
del n,n_mem,n_no

Members: 40238 (16.58%)
Non members: 202457 (83.42%)


In [ ]:
# # Undersample majority class
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(sampling_strategy= 4./6., replacement= False)
# rus_feat, rus_lab = rus.fit_resample(df[features], df[labels])
# rus_feat[labels] = rus_lab

# df = rus_feat
# del rus, rus_feat, rus_lab

# n_mem = df[df.member == 1].shape[0]
# n_no = df[df.member == 0].shape[0]
# n = df.shape[0]
# print ('Members: {} ({:.2f}%)'.format(n_mem, n_mem/n*100))
# print ('Non members: {} ({:.2f}%)'.format(n_no, n_no/n*100))
# del n,n_mem,n_no

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.35, stratify = df[labels])
print ('Training: {} members, {} non members'.format(train[train.member == 1].shape[0], train[train.member == 0].shape[0]))
print ('Testing: {} members, {} non members'.format(test[test.member == 1].shape[0], test[test.member == 0].shape[0]))

Training: 26155 members, 131596 non members
Testing: 14083 members, 70861 non members


### Model & Searches

In [ ]:
# from sklearn.svm import SVC
# model = SVC(probability= False, class_weight= 'balanced', verbose = False)

# params = dict(
#     C = [0.1, 1., 10],
#     kernel = ['linear', 'poly', 'rbf', 'sigmoid'],
#     degree = [2,3,4,7],
#     gamma = ['scale', 0.005, 0.1, 1.],
#     coef0 = [0, 1, -1],
#     shrinking = [True, False], 
# )

# from sklearn.model_selection import RandomizedSearchCV

# rs = RandomizedSearchCV(estimator= model, param_distributions= params, cv= 3, n_jobs= -1,  verbose= 4, n_iter = 100)
# rs.fit(df[features], df[labels])

# df_rs = pd.DataFrame(rs.cv_results_)

In [6]:
from sklearn.svm import SVC
model = SVC(probability= True, class_weight= 'balanced', verbose = False, 
            C = 1.0, kernel= 'linear', degree = 1, gamma = 'scale', coef0 = 0, shrinking= True)

model.fit(df[features], df[labels])

### Tests


In [ ]:
# from sklearn.metrics import roc_curve, auc, precision_recall_curve, classification_report

# pred = model.predict(test[features])
# scores = model.predict_proba(test[features])
# model_score = model.score(test[features], test[labels])

# fpr, tpr, thresholds_roc = roc_curve(test[labels], scores[:,1], pos_label=1)
# precision, recall, thresholds_pr = precision_recall_curve(test[labels], scores[:,1], pos_label= 1)

# print(f'Model score: {model_score}')
# print('ROC AUC: {}'.format(auc(fpr, tpr)))
# print('Precision-recall AUC: {}'.format(auc(recall, precision)))
# print('-'*70)
# print(classification_report(test[labels],pred))

In [ ]:
# from sklearn.metrics import confusion_matrix
# plt.figure(figsize=(16,4))

# # Confusion matrix
# plt.subplot(1, 3, 1)
# conf_m = confusion_matrix(test[labels], pred)
# df_conf_m = pd.DataFrame(conf_m, index=[0,1], columns=[0,1])
# sns.heatmap(df_conf_m, cmap=sns.color_palette('light:teal', as_cmap=True), annot=True, fmt='d')
# plt.xlabel('Predicted')
# plt.ylabel('True')

# # ROC Curve
# plt.subplot(1, 3, 2)
# plt.plot(fpr, tpr)
# plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('FPR')
# plt.ylabel('TPR')
# plt.grid()

# # Precision-recall curve
# plt.subplot(1, 3, 3)
# plt.plot(recall, precision)
# plt.plot([0, 1], [len (test[test.member == 1])/len(test), len (test[test.member == 1])/len(test)] , color='gray', linestyle='--')
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.grid()
# plt.xlim([0,1])
# plt.ylim([0,1])

# plt.show()

In [ ]:
importances = np.sort(model.feature_importances_)[::-1]
sorted_features = [x for _,x in sorted(zip(model.feature_importances_, features))][::-1]

plt.figure(figsize= (17, 10))
plt.grid()
sns.barplot(x = importances, y = sorted_features)